# Part 2 - Using a Class and Its Methods

## Exercising methods of the `Fuel` class

In this section, we’ll move from testing single equations to testing behaviors of class objects.

We’ll use the `Fuel` class, `FireParams`, and `FireEquations` from our source code. We will also introduce our "synthetic" fuels, which can be used to quickly mock up fuel objects that are based in reality, but have known conditions for more straightforward testing.

---

## Goals

* Instantiate `Fuel` objects based in reality, but with basic, standardized characteristics
* Test methods of the `Fuel` class
* Check qualitative properties like monotonicity, bounds, and trends.
* Visualize outputs to spot unexpected behavior.
* Illustrate separation of concerns for better testability.
  
---

## Key Concepts

* **Synthetic inputs**: Create simple, controlled examples to explore expected behavior.

---

## Exercise 2.1: Set up `Fuel` object

Check out the code in the `fuel_class.py` file. This file contains our `Fuel` class:

```python
class Fuel:
    """Represents fuel characteristics of a patch of land"""

    def __init__(self, params: FireParams) -> None:
        """Initialize a new Fuel instance.

        Args:
            params (FireParams): parameters
        """
        self.params = params
        self.loading = np.zeros(NUM_FUEL_CLASSES, dtype=float)
        self.effective_moisture = np.zeros(NUM_FUEL_CLASSES, dtype=float)
        self.frac_loading = np.zeros(NUM_FUEL_CLASSES, dtype=float)
        self.frac_burnt = np.zeros(NUM_FUEL_CLASSES, dtype=float)
        self.non_trunk_loading = 0.0
        self.average_moisture_notrunks = 0.0
        self.bulk_density_notrunks = 0.0
        self.sav_notrunks = 0.0
        self.mef_notrunks = 0.0
```

This class houses characteristics of fuel for a patch of land that we are interested in, including amount of fuel (`loading`, kgC m$^{-2}$), moisture (`effective_moisture`, m$^3$ m$^{-3}$), fraction burnt (`frac_burnt`), bulk density (`bulk_density_notrunks`, kg m$^{-3}$), and surface area to volume ratio (`sav_no_trunks`, cm$^{-1}$).

There are also methods for the `Fuel` class, like `update_loading`, `compute_moisture`, and `average_bulk_density_no_trunks`.

### More about fuel

In the Rothermel fuel model, surface fuel is broken up by size, as this is one of the primary characteristics that effects fire behaviour:

#### Fuel classes
1. "1-hour" fuels (i.e., dead leaves)
2. "10-hour" fuels (i.e., twig litter)
3. "100-hour" fuels (i.e., branch litter)
4. "1000-hour" fuels (i.e., downed trunks)
5. live herbaceous fuel (i.e., grasses)

In the FATES code, we additionally break the branch litter up into small and large branches, thus representing six total fuel classes. Notice that some of the `Fuel` object attributes are (size 6) arrays, but some are scalars. The arrays hold values for each fuel class, whereas the scalars represent weighted-averages across the fuel classes.

<div class="alert alert-block alert-info"><b>Note: </b> Trunks are not used to calculate rate of spread, but we are interested in how much they may burn. As such, our weighted-average scalars do not include trunks in the averaging, but we still want to capture information about them in our arrays.</div>

---

### Instatiating the `Fuel` object

Alright, let's go ahead and instantiate an instance of the `Fuel` class. We need parameter values from our `FireParams` class. We can just go head and read in the default parameter file.

In [1]:
from fire_model.fire_params import FireParams
from fire_model.fuel_class import Fuel

# read in parameter file
params = FireParams.from_yaml("../parameter_files/fire_parameters.yaml")

# instantiate a fuel object
fuel = Fuel(params)

# print out the attributes of the fuel instance
fuel.describe()

Fuel instance state:
  Loading (kgC/m2): [0. 0. 0. 0. 0. 0.]
  Effective moisture (m3/m3): [0. 0. 0. 0. 0. 0.]
  Fractional loading: [0. 0. 0. 0. 0. 0.]
  Fraction burnt: [0. 0. 0. 0. 0. 0.]
  Non-trunk loading (kgC/m2): 0.0000
  Avg moisture (non-trunks, m3/m3): 0.0000
  Bulk density (non-trunks, kg/m3): 0.0000
  SAV (non-trunks, /cm): 0.0000
  MEF (non-trunks, m3/m3): 0.0000


Well, this is not very interesting or useful. By design, the `Fuel` objects are instantiated with all zeros. If we try to call its methods, we will probably get really unintersting (and unhelpful) results.